In [1]:
using Random
using LinearAlgebra

function euclidean_norm(vec::Vector)
    return sqrt(sum(vec.^2))
end

function generate_symmetric_matrix(l::Int, r::Int, n::Int)    
    A = rand(n, n) .* (r - l) .+ l
    A = (A + A') / 2
    return A
end

function identity_matrix(n::Int)    
    return Matrix{Float64}(I, n, n)
end

# Пример использования
n = 3
symmetric_matrix = generate_symmetric_matrix(-10,10,n)

println("Симметрическая матрица:")
println(symmetric_matrix)

println(identity_matrix(3))

Симметрическая матрица:
[8.046443861991847 2.0390193850530123 7.685887410277907; 2.0390193850530123 -1.126120869331599 6.109143995229385; 7.685887410277907 6.109143995229385 5.930253743360591]
[1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0]


In [2]:
function danilevsky_algo(matrix::Matrix)
    n = size(matrix,1)
    Bs = identity_matrix(n)
    D = copy(matrix)

    for i in n:-1:2
        B = identity_matrix(n)
        # заполняем B
        d = D[i, i - 1]
        B[i-1,1:end] = - D[i,1:end] ./ d
        B[i - 1,i - 1] = 1 / d
        # B^(-1)
        B_inv = inv(copy(B))
        # процесс получения матрицы P и произведениие B_i
        D = B_inv * D * B
        Bs = Bs * B
    end
    
    return Bs, D
end

danilevsky_algo (generic function with 1 method)

In [3]:
function krylov_algo(matrix::Matrix)
    p = []
    n = size(matrix, 1)
    D = copy(matrix)
    y = ones(n+1,n)
    A = zeros(n,n)
    for i in 2:n+1
        y[i,1:end] = D*y[i - 1,1:end]
    end
    for i in n:-1:1
        A[n-i+1,1:end]=y[i,1:end]
    end
    A = A'
    f = y[n+1,1:end]
    res_p = A\f
    
    p = [1.0]
    for pp in res_p
        push!(p,-pp)
    end
    
    return y, p
end

A = [1.0 2.0 3.0; 4.0 5.0 6.0; 7.0 8.0 9.0]
B, P = krylov_algo(A)

println("Приведенная матрица B:")
println(B)
println("Матрица преобразования P:")
println(P)

Приведенная матрица B:
[1.0 1.0 1.0; 6.0 15.0 24.0; 108.0 243.0 378.0; 1728.0 3915.0 6102.0]
Матрица преобразования P:
[1.0, -0.015873015873009256, -242.76190476190484, -269.71428571428584]


In [4]:
function union_intervals(ints::Vector)
    union = []
    ints  = sort(copy(ints), by=x->x[1])
    for int in ints
        if length(union)>0 && union[end][2]>=int[1]-1
           union[end][2] = max(union[end][2],int[2])
        else
            push!(union,[int[1],int[2]])
        end
    end

    return union
end

function gershgorin_intervals(A::Matrix)
    n = size(A, 1)
    intervals = []
    
    for i = 1:n
        radius = sum(abs.(A[i, :])) - abs(A[i, i])
        center = A[i, i]
        push!(intervals, (center - radius, center + radius))
    end

    intervals = union_intervals(copy(intervals))
    
    return intervals
end

A = [1.0 2.0 3.0; 4.0 5.0 10.0; 7.0 8.0 9.0]
intervals = gershgorin_intervals(A)

println("Интервалы Гершгорина для матрицы A:")
intervals

Интервалы Гершгорина для матрицы A:


1-element Vector{Any}:
 [-9.0, 24.0]

In [17]:
function find_equatation_coeffs(P::Matrix)
    equatationCoeffs = copy(P[1,1:end]).*(-1)
    equatationCoeffs = [1.0; equatationCoeffs]
    
    return equatationCoeffs
end

function polynomy(a::Vector, x)
    val = 0
    n = length(a)
    for i in n-1:-1:0
        val += a[n - i] * (x^i)
    end
    
    return val
end


function find_eigen_values(eqCoeffs::Vector, intervals::Vector, len)
    values = []
    len2 = 10e-7
    for interval in intervals
        left = interval[1]
        right = interval[2]
        l = Int(floor((right - left) / len))
    
        for i in 0:l
            x_left = left + i * len
            x_right = x_left + len
            y_left = polynomy(eqCoeffs, x_left)
            y_right = polynomy(eqCoeffs, x_right)
            alpha = y_left * y_right

            if alpha < 0
                while x_right - x_left >= len2
                    x_middle = (x_right + x_left) / 2
                    y_middle = polynomy(eqCoeffs, x_middle)
                    beta = y_left * y_middle
                    if beta < 0
                        x_right = x_middle
                    else
                        x_left = x_middle
                    end
                end
                push!(values,(x_right + x_left) / 2)
            elseif y_left == 0
                push!(values,x_left)
            elseif y_right == 0
                push!(values,x_right)
            end
        end
    end
    
    return values
end


function find_eigen_vectors_krylov(y, vals, p)
    n = length(p) - 1
    xs = zeros(n,n)
    for i in 1:n
        x = y[n,1:end]
        q_i = [1.0]
        for j in 2:n
            push!(q_i,vals[i] * q_i[j - 1] + p[j])
            x = x + q_i[j]* y[n - j+1,1:end]
        end
        xs[i,1:end]= x./ euclidean_norm(x)
    end

  return xs
end

function find_eigen_vectors_danilevsky(vals, B)
    n = size(B,1)
    m = length(vals)
    ys = zeros(m,n)
    for i in 1:m
        for j in n-1:-1:0
          ys[i, n - j] = vals[i] ^ j
        end
    end
    # собственные вектора
    xs = zeros(size(ys,1), size(ys,2))
    for i in 1:size(ys,1)
        xs[i,1:end] = B * ys[i,1:end] 
        # нормализация
        xs[i,1:end] = xs[i,1:end] ./ euclidean_norm(xs[i,1:end])
    end
    
    return xs
end

find_eigen_vectors_danilevsky (generic function with 1 method)

In [6]:
function check_vieta(A::Matrix, vals::Vector)
    sum_eigs=sum(vals)
    sp = 0
    n = size(A,1)
    for i in 1:n
        sp += A[i,i]
    end
    if abs(sum_eigs-sp)>0.1
        println("\nVieta`s theorem doesn`t work")
    else
        println("\nVieta`s theorem works")
    end
end

function check_gershorin(vals::Vector, intervals::Vector)
    for interval in intervals
        for val in vals
            if val > interval[2] || val < interval[1]
                println("Gershgorin`s theorem error\n") 
                return
            end
        end
    end

    println("Gershgorin`s theorem works\n")
end

function check_ortogonal(n::Int, vecs::Matrix)
    for i in 1:n-1
        for j in i + 1:n
            scal = dot(vecs[i,1:end], vecs[j,1:end])
            if abs(scal) > 0.1
                println("Eigen vectors are not orthogonal\n")
                println(abs(scal))
                return
            end
        end
    end
        
    println("\nEigen vectors are orthogonal")
end

check_ortogonal (generic function with 1 method)

In [21]:
n = 4
A = [2.2 1 0.5 2; 1 1.3 2 1; 0.5 2 0.5 1.6; 2 1 1.6 2]
println("matrix: $A\n")

intervals = gershgorin_intervals(A)
println("intervals: $intervals\n")

println("\nKRYLOV")
Ys, P = krylov_algo(A)
println("P: $P\n")
println("Y: $Ys\n")


eig_vals = find_eigen_values(P,intervals,10e-3)
println("eigvals: $eig_vals")

check_vieta(A,eig_vals)
check_gershorin(eig_vals, intervals)

# julia library
# eig_vects = eigvects(P)
# println("eigvects: $eig_vects")

eig_vects = find_eigen_vectors_krylov(Ys,eig_vals, P)
for i in 1:size(eig_vects,1)
    vec = eig_vects[i,1:end]
    println("eigvect_$i: $vec")
end

check_ortogonal(n,eig_vects)


println("\nDANILEVSKY")
B, P = danilevsky_algo(A)
println("P: $P\n")
println("B: $B\n")

equatationCoeffs = find_equatation_coeffs(P)
println("equatation coeffs: $equatationCoeffs\n")

# julia library
eig_vals = eigvals(P)
println("eigvals: $eig_vals")

eig_vals = find_eigen_values(equatationCoeffs,intervals,10e-3)
println("eigvals: $eig_vals")

check_vieta(A,eig_vals)
check_gershorin(eig_vals, intervals)

# julia library
# eig_vects = eigvects(P)
# println("eigvects: $eig_vects")

eig_vects = find_eigen_vectors_danilevsky(eig_vals, B)
for i in 1:size(eig_vects,1)
    vec = eig_vects[i,1:end]
    println("eigvect_$i: $vec")
end

check_ortogonal(n,eig_vects)

matrix: [2.2 1.0 0.5 2.0; 1.0 1.3 2.0 1.0; 0.5 2.0 0.5 1.6; 2.0 1.0 1.6 2.0]

intervals: Any[[-3.5999999999999996, 6.6]]


KRYLOV
P: [1.0, -5.999999999999894, -0.2000000000005567, 12.734999999999483, -2.761599999998538]

Y: [1.0 1.0 1.0 1.0; 5.7 5.3 4.6 6.6; 33.34 28.39 26.31 37.26; 189.413 160.12699999999998 146.221 211.68599999999998; 1073.3181 901.7060999999999 826.7686 1196.2786]

eigvals: Any[-1.4200863647460937, 0.22263580322265686, 1.5454183959960939, 5.652032165527345]

Vieta`s theorem works
Gershgorin`s theorem works

eigvect_1: [-0.222042593227019, 0.5159106786075803, -0.7572739835488279, 0.33327071928355184]
eigvect_2: [0.5219206423771644, 0.45486963802490893, -0.15344683758462913, -0.7050861816111366]
eigvect_3: [0.6289298182269153, -0.5725741727668471, -0.48565374919670573, 0.20185771304709482]
eigvect_4: [0.5317360709371982, 0.44619411921045116, 0.40881553103618407, 0.5924841098543349]

Eigen vectors are orthogonal

DANILEVSKY
P: [6.0 0.20000000000000284 -12.7350000000000

In [19]:
n = 10
A = generate_symmetric_matrix(-10,10,n)

println("\KRYLOV")
intervals = gershgorin_intervals(A)
Ys, P = krylov_algo(A)


eig_vals = find_eigen_values(P,intervals,10e-3)
println("eigvals: $eig_vals")

len = size(eig_vals,1)
if len<n
    println("\nfailed to find all eig_val, found $len")
else
    check_vieta(A,eig_vals)
    check_gershorin(eig_vals, intervals)
    
    eig_vects = find_eigen_vectors_krylov(Ys,eig_vals, P)
    println(size(eig_vects,1))
    for i in 1:size(eig_vects,1)
        vec = eig_vects[i,1:end]
        println("eigvect_$i: $vec")
    end

    check_ortogonal(n,eig_vects)
end

println("\nDANILEVSKY")
B, P = danilevsky_algo(A)

equatationCoeffs = find_equatation_coeffs(P)

eig_vals = find_eigen_values(equatationCoeffs,intervals,10e-3)
println("eigvals: $eig_vals")

len = size(eig_vals,1)
if len<n
    println("\nfailed to find all eig_val, found $len")
else
    check_vieta(A,eig_vals)
    check_gershorin(eig_vals, intervals)
    
eig_vects = find_eigen_vectors_danilevsky(eig_vals, B)
println(size(eig_vects,1))
for i in 1:size(eig_vects,1)
    vec = eig_vects[i,1:end]
    println("eigvect_$i: $vec")
end

    check_ortogonal(n,eig_vects)
end

eigvals: Any[-21.694629803531676, -19.904392987125426, -16.195854168766054, -7.119602948062923, -1.98592191778949, 1.8037411681480093, 5.477457598812075, 9.09046846306989, 16.10252046502302, 17.593263873226142]

Vieta`s theorem works
Gershgorin`s theorem works

10
eigvect_1: [-0.21666821816143766, -0.3097486582856601, 0.24090581281772136, 0.42184458542641873, -0.2433582911676938, -0.08762876114639499, -0.6494053387747015, 0.24119282553578952, -0.25909793041220813, -0.08477687599223555]
eigvect_2: [0.5772926333312461, -0.5554372762458667, 0.10048715303554998, 0.29654856029930154, 0.20211726102217648, -0.13167304069239913, 0.07578216160955542, 0.03705326332527105, 0.3876577521884709, 0.2111879283858198]
eigvect_3: [0.3766288160862275, -0.06984085035878544, -0.2400098937671476, -0.2922062109012542, -0.40177446117673826, 0.009818033189374742, 0.05073464572740252, 0.5917958048854338, 0.012210436411351595, -0.44251473034833283]
eigvect_4: [-0.3304571704240979, 0.15008600964890004, -0.0001464